<a href="https://colab.research.google.com/github/TiagoBarantini/Pro_Eng_dados/blob/main/prepara%C3%A7%C3%A3o%2C_orquestra%C3%A7%C3%A3o_e_fluxos_de_dados_aula04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/campeonato-brasileiro-cartoes.csv
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/campeonato-brasileiro-estatisticas-full.csv
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/campeonato-brasileiro-full.csv
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/campeonato-brasileiro-gols.csv

In [ ]:
# Instalação Java
%%bash
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Instalação do PySpark
pip install -q pyspark

In [6]:
#definir uma variavel de ambiente para identificador local do JAVA
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [7]:
#Definir a sessão spark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession.builder.getOrCreate()

In [8]:
param = {
    "header": True,
    "inferSchema":True,
    "sep": ","
}

cartoes = (
    spark
    .read
    .csv("campeonato-brasileiro-cartoes.csv", **param)
)

gols = (
    spark
    .read
    .csv("campeonato-brasileiro-gols.csv", **param)
)

estats = (
    spark
    .read
    .csv("campeonato-brasileiro-estatisticas-full.csv", **param)
)

In [11]:
schemafull = """
  ID int, rodada int, data string, hora string, dia string, mandante string, visitante string,
  formacao_mandante string, formacao_visitante string, tecnico_mandante string, tecnico_visitante string,
  vencedor string, arena string, mandante_placar int, visitante_placar int, mandante_estado string, visitante_estado string,
  estado_vencedor string
"""


full = (
    spark
    .read
    .csv("campeonato-brasileiro-full.csv",header=True, sep=",", schema=schemafull)
)

full.select(
    "ID", "rodada","vencedor","data", "hora","dia"
).show()

+---+------+-------------+----------+-----+-------+
| ID|rodada|     vencedor|      data| hora|    dia|
+---+------+-------------+----------+-----+-------+
|  1|     1|      Guarani|2003-03-29|16:00| Sabado|
|  2|     1| Athletico-PR|2003-03-29|16:00| Sabado|
|  3|     1|            -|2003-03-30|16:00|Domingo|
|  4|     1|            -|2003-03-30|16:00|Domingo|
|  5|     1|            -|2003-03-30|16:00|Domingo|
|  6|     1|     Criciuma|2003-03-30|16:00|Domingo|
|  7|     1|            -|2003-03-30|16:00|Domingo|
|  8|     1|            -|2003-03-30|16:00|Domingo|
|  9|     1|            -|2003-03-30|16:00|Domingo|
| 10|     1|            -|2003-03-30|16:00|Domingo|
| 11|     1|            -|2003-03-30|18:00|Domingo|
| 12|     1|  Atletico-MG|2003-03-30|18:00|Domingo|
| 13|     2|            -|2003-04-05|16:00| Sabado|
| 14|     2|            -|2003-04-05|16:00| Sabado|
| 15|     2|Internacional|2003-04-05|16:00| Sabado|
| 16|     2|       Gremio|2003-04-05|18:00| Sabado|
| 17|     2|

In [ ]:
%%bash
curl -O https://github.com/neylsoncrepalde/campeonato_brasileiro_dataset/blob/main/Legenda.txt
cat Legenda.txt

In [15]:
gols.createOrReplaceTempView("gols")
estats.createOrReplaceTempView("estats")
full.createOrReplaceTempView("full")
cartoes.createOrReplaceTempView("cartoes")

#Em quantas partidas o palmeiras recebeu cartões amarelos quando jogava como visitante?

In [ ]:
cartoes.where("clube = 'Palmeiras' AND cartao = 'Amarelo'").show()

In [ ]:
full.where("visitante = 'Palmeiras'").show()

In [52]:
(
    full
    .where("visitante = 'Palmeiras'")
    .select('ID', 'visitante')
    .join(cartoes, full.ID == cartoes.partida_id,"inner")
    .where("clube = 'Palmeiras' AND cartao = 'Amarelo'")
    .select('ID')
    .distinct()
    .agg(
        f.count('ID').alias('CONTAGEM')
    )
).show()

+--------+
|CONTAGEM|
+--------+
|     143|
+--------+



In [29]:
spark.sql("""
    SELECT
      count(1) as contagem
    FROM(
      SELECT distinct
        a.ID
      FROM full a
      INNER JOIN cartoes b
        ON a.ID = b.partida_id
      WHERE a.visitante = 'Palmeiras' AND b.clube = 'Palmeiras' AND b.cartao = 'Amarelo')
""").show()

+--------+
|contagem|
+--------+
|     143|
+--------+

